# Imports

In [1]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [2]:
events = pd.read_pickle("../../../../data/tp2/events_tp2_formateado.pkl")

In [3]:
targets_competencia = pd.read_pickle("../../../../data/tp2/ref_hashes_target.pkl")

---

## Etiqueto eventos más frecuentes

In [4]:
EVENTOS_MAS_FRECUENTES = [15,23,7,0,2]

In [5]:
# Etiqueto eventos
for evento in EVENTOS_MAS_FRECUENTES:
    events["es_evento_{}".format(evento)] = (events["event_id"] == evento).astype(np.int8)

### De events, me quedo con sólo datos que correspondan a la competencia

In [6]:
events_competencia = events.loc[events["device_id"].isin(targets_competencia.index)]

---

# Cargo features generados hasta el momento

## Sets de entrenamiento

In [7]:
# Ventanas de tres dias
Xs = {}
try:
    print("Cargando features")
    for ventana_nro in range(1,5):    
        Xs[ventana_nro] = pd.read_csv("../../features/ventana_{}_entrenar_events.csv".format(ventana_nro), index_col=0)
except FileNotFoundError:
    print("No se encontraron los features, se generarán los dataframe requeridos")
    for ventana_nro in range(1,5):    
        Xs[ventana_nro] = events.loc[events["ventana_{}".format(ventana_nro)]]["device_id"] \
            .drop_duplicates().to_frame().set_index("device_id").copy()

Cargando features
No se encontraron los features, se generarán los dataframe requeridos


## Set que se usará para predecir

In [8]:
# Features para hacer las predicciones que se suben, corresponden a la ventana 7
Xs_predecir = {}

try:
    print("Cargando features usados para predecir")
    Xs_predecir[7] = pd.read_csv("../../features/predecir_events.csv", index_col=0)
except FileNotFoundError:
    print("No se encontraron los features usados para predecir, se generará el dataframe requerido")
    Xs_predecir[7] = targets_competencia.copy()

Cargando features usados para predecir
No se encontraron los features usados para predecir, se generará el dataframe requerido


## Diccionario de features

In [9]:
features = {}
# Todos los datasets tienen la misma cantidad y nombre de features, tomo la ventana 1
columnas_ya_generadas = list(Xs[1].columns.values)

---

# Creación de features

## Cantidad de apariciones en eventos dentro de la ventana

In [10]:
def cantidad_apariciones_en_ventana(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : "count"}).rename(columns={"date": "cantidad_apariciones_en_ventana_events"})

In [11]:
features["cantidad_apariciones_en_ventana"] = (cantidad_apariciones_en_ventana, ["cantidad_apariciones_en_ventana_events"])

---

## Cantidad de apariciones por período horario

In [12]:
def cantidad_apariciones_por_periodo_horario(dataframe, nro_ventana):
    features = dataframe.groupby("device_id").agg( \
       {"hora_2_a_6" : "sum",
        "hora_6_a_11" : "sum",
        "hora_11_a_14" : "sum",
        "hora_14_a_18" : "sum",
        "hora_18_a_23" : "sum",
        "hora_23_a_2" : "sum"})
    features.columns = ["apariciones_" + x + "_events" for x in features.columns]
    return features

In [13]:
columnas_generadas = ["apariciones_" + x + "_events" for x in ["hora_2_a_6", "hora_6_a_11", "hora_11_a_14", "hora_14_a_18", "hora_18_a_23", "hora_23_a_2"]]
features["cantidad_apariciones_por_periodo_horario"] = (cantidad_apariciones_por_periodo_horario, columnas_generadas)

---

## Cantidad de apariciones según en qué dia de la ventana se encuentra

In [14]:
def cantidad_apariciones_por_dia(dataframe, nro_ventana):
    apariciones_por_dia = dataframe.groupby(["device_id", "dia"]).agg(({"date" : "count"}))
    apariciones_por_dia = apariciones_por_dia.unstack().droplevel(level=0, axis=1)
    cantidad_de_dias = len(apariciones_por_dia.columns)
    nuevas_columnas = []
    for dia in range(cantidad_de_dias):
        nuevas_columnas.append(dia+1)
    apariciones_por_dia.columns = nuevas_columnas
    for dia in range(1, 4):
        if dia not in list(apariciones_por_dia.columns.values):
            apariciones_por_dia.insert(dia, dia, 0)
    apariciones_por_dia.columns = ["apariciones_en_dia_{}_events".format(dia) for dia in range(1,4)]
    return apariciones_por_dia

In [15]:
columnas_generadas = ["apariciones_en_dia_{}_events".format(dia) for dia in range(1,4)]
features["cantidad_apariciones_por_dia"] = (cantidad_apariciones_por_dia, columnas_generadas)

---

## Promedio de eventos por día dentro de la ventana

In [16]:
def promedio_eventos_por_dia(dataframe, nro_ventana):
    feature = dataframe.groupby(["device_id","dia_de_la_semana"]).agg({"date":"count"})
    feature = feature.groupby("device_id").agg({"date":"mean"})
    return feature.rename(columns={"date": "promedio_eventos_por_dia_events"})

In [17]:
features["promedio_eventos_por_dia"] = (promedio_eventos_por_dia, ["promedio_eventos_por_dia_events"])

---

## Cantidad de veces que cada dispositivo realizó su evento más frecuente

In [18]:
def cantidad_realizaciones_evento_mas_frecuente(dataframe, nro_ventana):
    feature = events.groupby(["device_id","event_id"]).agg({"date":"count"})
    feature = feature.groupby("device_id").agg({"date":"max"})
    feature.rename(columns = {"date":"cantidad_realizaciones_evento_mas_frecuente_events"},inplace = True)    
    return feature    

In [19]:
features["cantidad_realizaciones_evento_mas_frecuente"] = (cantidad_realizaciones_evento_mas_frecuente, ["cantidad_realizaciones_evento_mas_frecuente_events"])

---

## Cantidad de eventos top para cada dispositivo

In [20]:
def cantidad_eventos_top(dataframe, nro_ventana):
    feature = events.groupby(["device_id","event_id"]).agg({"date":"count"})
    idx = feature.groupby(['device_id'])["date"].transform(max) == feature["date"]
    feature = feature[idx]
    feature = feature.groupby("device_id").agg({"date":"count"})
    feature.rename(columns = {"date":"cantidad_eventos_top_events"},inplace = True)    
    return feature

In [21]:
features["cantidad_eventos_top"] = (cantidad_eventos_top, ["cantidad_eventos_top_events"])

---

## Tiempo desde hora más frecuente del dispositivo hasta las 0 horas

In [22]:
def tiempo_desde_hora_mas_frecuente_hasta_00(dataframe, nro_ventana):
    segundos_en_hora = 60*60    
    feature = dataframe.groupby(["device_id","hora"]).agg({"date":"count"})
    idx = feature.groupby(["device_id"])["date"].transform(max) == feature["date"]
    feature = feature[idx]
    
    feature = feature.reset_index().groupby("device_id").agg({"hora":"min"})
    feature["hora"] = feature["hora"] * segundos_en_hora
    feature.rename(columns = {"hora":"tiempo_desde_hora_mas_frecuente_hasta_00_events"},inplace = True)    
    return feature    

In [23]:
features["tiempo_desde_hora_mas_frecuente_hasta_00"] = (tiempo_desde_hora_mas_frecuente_hasta_00, ["tiempo_desde_hora_mas_frecuente_hasta_00_events"])

---

## Cantidad de eventos con y sin wifi

In [24]:
def cantidad_eventos_con_sin_wifi(dataframe, nro_ventana):
    feature = dataframe[["device_id", "wifi", "dia"]].groupby(["device_id", "wifi"]).agg({"dia" : "count"}).unstack()
    feature.columns = ["cantidad_eventos_con_wifi_events", "cantidad_eventos_sin_wifi_events"]
    return feature

In [25]:
columnas_generadas = ["cantidad_eventos_con_wifi_events", "cantidad_eventos_sin_wifi_events"]
features["cantidad_eventos_con_sin_wifi"] = (cantidad_eventos_con_sin_wifi, columnas_generadas)

---

## Para los eventos top, cuántas veces el dispositivo realizó cada evento top

In [26]:
agregacion = {}
for evento in EVENTOS_MAS_FRECUENTES:
    agregacion["es_evento_{}".format(evento)] = "sum"

In [27]:
def cantidad_realizaciones_eventos_top(dataframe, nro_ventana):
    feature = dataframe.groupby("device_id").agg(agregacion)
    feature.columns = ["cantidad_evento_{}_events".format(x) for x in EVENTOS_MAS_FRECUENTES]
    return feature

In [28]:
features["cantidad_realizaciones_eventos_top"] = (cantidad_realizaciones_eventos_top, ["cantidad_evento_{}_events".format(x) for x in EVENTOS_MAS_FRECUENTES])

---

# Genero los features

## Función generadora de features en ventanas

In [29]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos, ventana_inicial, ventana_final):
    """El rango se toma como python, [ventana_inicial, ventana_final)"""
    for ventana_nro in range(ventana_inicial, ventana_final):
        feature = generador_feature(dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]], ventana_nro)
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True, how="left")

### Genero los features

In [30]:
COLUMNAS_QUE_GENERA_EL_FEATURE = 1
FUNCION_PARA_CALCULAR_EL_FEATURE = 0

for nombre_feature in features:    
    columnas_generadas = features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE]
    feature_ya_generado = False
    for columna in columnas_generadas:
        if columna in columnas_ya_generadas:
            feature_ya_generado = True
            break
    
    if feature_ya_generado:
        continue

    generar_feature_en_ventanas(events, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs, 1, 5)
    generar_feature_en_ventanas(events_competencia, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs_predecir, 7, 8)    
    columnas_ya_generadas.extend(features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE])

---

# Guardado de los sets de entrenamiento y el set de prediccion

In [31]:
for ventana_nro in range(1, 5):
    Xs[ventana_nro].to_csv("../../features/ventana_{}_entrenar_events.csv".format(ventana_nro))

Xs_predecir[7].to_csv("../../features/predecir_events.csv")

---